In [ ]:
%pip install selenium parsel

^C
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\anfiniti\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # Import Service untuk mendefinisikan path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import pandas as pd
from IPython.display import clear_output

# Buat list untuk menyimpan data
data = []

# Tentukan path ke Chromedriver
chromedriver_path = './chromedriver.exe'  # Ubah sesuai lokasi file Anda

# Buat objek Service dengan path driver
service = Service(chromedriver_path)

# Opsi untuk membuka Chrome dalam bahasa Indonesia dan menonaktifkan pemuatan gambar
options = webdriver.ChromeOptions()
options.add_argument("--lang=id")
# options.add_argument("--blink-settings=imagesEnabled=false")  # Menonaktifkan pemuatan gambar
# options.add_argument("--headless")  # Tambahkan jika ingin menjalankan secara headless
# options.add_argument("--disable-gpu")  # Untuk meningkatkan kompatibilitas headless
options.add_argument("--blink-settings=imagesEnabled=false")  # Menonaktifkan pemuatan gambar
# Inisialisasi webdriver dengan opsi
driver = webdriver.Chrome(service=service, options=options)

# Buka halaman web
driver.get("https://maps.app.goo.gl/j3JsbaJDZqvHpXS49")

time.sleep(5)
# Tunggu elemen tombol "Lihat Semua Ulasan" muncul
try:

    see_all_reviews_button = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, '//button[@role="tab" and contains(@aria-label, "Ulasan untuk")]'))
    )
    see_all_reviews_button.click()

except Exception as e:
    print("Tombol 'Lihat Semua Ulasan' tidak ditemukan")
    driver.quit()
    raise
time.sleep(5)
try:
    urutkan = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, '//button[contains(@aria-label, "Urutkan ulasan")]'))
    )
    urutkan.click()

except Exception as e:
    print("Tombol 'Urutkan' tidak ditemukan:")
    driver.quit()
    raise
time.sleep(5)
try:
    terbaru = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="fxNQSd" and @data-index="1"]'))
    )
    terbaru.click()

except Exception as e:
    print("Tombol 'Terbaru' tidak ditemukan:")
    driver.quit()
    raise

def scroll_to_load_reviews():
    try:
        # Tunggu hingga elemen scrollable muncul
        scrollable_div = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="m6QErb DxyBCb kA9KIf dS8AEf XiKgde "]'))
        )

        # Ambil jumlah ulasan
        jumlah_ulasan_element = driver.find_element(By.XPATH, '//div[@class="fontBodySmall" and contains(text(), "ulasan")]')
        jumlah_ulasan_text = jumlah_ulasan_element.text.split()[0]
        jumlah_ulasan = int(jumlah_ulasan_text.replace('.', ''))
        print('Jumlah Ulasan:', jumlah_ulasan)

        while True:
            # Scroll ke bawah
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            jumlah_username = len(driver.find_elements(By.XPATH, './/div[@class="d4r55 "]'))
            clear_output(wait=True)
            print('Jumlah Ulasan:', jumlah_username)

            if jumlah_username == jumlah_ulasan: # ini update jumlahnya sesuai dengan gmaps yaa
                print("Reached the bottom of the reviews.")
                break

    except Exception as e:
        print(f"Error during scrolling: {e}")

# Scroll untuk memuat review
scroll_to_load_reviews()

# Ambil review
reviews = driver.find_elements(By.XPATH, '//div[@class="jftiEf fontBodyMedium "]')
stars = driver.find_elements(By.XPATH, '//span[@class="kvMYJc"]')

# Proses data
for review, star in zip(reviews, stars):
    try:
        username = review.find_element(By.XPATH, './/div[@class="d4r55 "]').text
    except:
        username = "Anonymous"
    try:
        review_text = review.find_element(By.XPATH, './/span[@class="wiI7pd"]').text
    except:
        review_text = "No review text"
    try:
        star_count = star.get_attribute("aria-label").split()[0]  # Ambil angka bintang
    except:
        star_count = "0"

    # Tambahkan data ke list
    data.append([username, review_text, star_count])

# Tutup browser
driver.quit()

Jumlah Ulasan: 1260


KeyboardInterrupt: 

In [4]:
# Buat DataFrame dari data
df_reviews = pd.DataFrame(data, columns=['Name', 'content', 'score'])
df_reviews.to_csv('place_reviews.csv', index=False)

In [5]:
df_reviews2 = pd.read_csv('Safari Beach Batang reviews.csv')

In [6]:
print(len(df_reviews))
print(len(df_reviews2))

10
332


In [7]:
df_reviews.head()

,Name,content,score
0,akhmad luqmanul,Mantap sekali untuk liburan bersama keluarga u...,5
1,Indra Maulana,Puas liburan disana,5
2,shanum malayeka,"Bagus banget tempatnya, ada pentas lumba2nya j...",5
3,Permana permana,No review text,5
4,Michael Ryan,No review text,5
